In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import zipfile
import io
import time
from datetime import datetime
import pyquery
import thread
import os
from lxml import etree as ET
from collections import defaultdict

## Environment

In [3]:
from dwre_tools.env import get_default_project, get_project, load_config, get_environment
import dwre_tools

In [4]:
PROJECT = get_project("swk")
ENV = get_environment("dev01", PROJECT)

MIGRATIONS_DIR = "/Users/clavery/code/swk/swk-demandware-website/migrations/"

## Login Session

In [376]:
webdavsession = requests.session()
webdavsession.auth=(ENV["username"], ENV["password"],)
bmsession = requests.session()

In [377]:
bmsession.post("https://{}/on/demandware.store/Sites-Site/default/ViewApplication-ProcessLogin".format(ENV["server"]),
               data=dict(
                    LoginForm_Login=ENV["username"],
                    LoginForm_Password=ENV["password"],
                    LocaleID="",
                    LoginForm_RegistrationDomain="Sites",
                    login=""
                ))

<Response [200]>

## Utilities

In [332]:
def wait_for_import(session, filename):
    response = session.get("https://{}/on/demandware.store/Sites-Site/default/ViewSiteImpex-Status".format(ENV["server"]))
    response_q = pyquery.PyQuery( response.content)
    log_link = response_q.find("a:contains('Site Import'):contains('%s')" % filename).eq(0).attr("href")
    if not log_link:
        raise Exception("Failure to find status link for %s. Check import log." % filename)
    finished = False
    while not finished: 
        log_response = session.get(log_link)
        if "finished successfully" in log_response.content:
            finished = True
        elif "aborted" in log_response.content:
            raise Exception("Failure to import %s. Check import log." % filename)
        else:
            thread.sleep(2)

In [368]:
SCHEMA_MAP = {
    "abtest" : "abtest.xsd",
    "bmext" : "bmext.xsd",
    "cache-settings" : "cachesettings.xsd",
    "catalog" : "catalog.xsd",
    "coupons" : "coupon.xsd",
    "coupon-redemptions" : "couponredemption.xsd",
    "customers" : "customer.xsd",
    "customer-groups" : "customergroup.xsd",
    "customer-lists" : "customerlist.xsd",
    "customerpaymentinstrument" : "customerpaymentinstrument.xsd",
    "custom-objects" : "customobject.xsd",
    "feeds" : "feed.xsd",
    "form" : "form.xsd",
    "geolocations" : "geolocation.xsd",
    "gift-certificates" : "giftcertificate.xsd",
    "inventory" : "inventory.xsd",
    "library" : "library.xsd",
    "metadata" : "metadata.xsd",
    "oauth" : "oauth.xsd",
    "orders" : "order.xsd",
    "payment-settings" : "paymentmethod.xsd",
    "payment-processors" : "paymentprocessor.xsd",
    "preferences" : "preferences.xsd",
    "pricebooks" : "pricebook.xsd",
    "product-lists" : "productlist.xsd",
    "promotions" : "promotion.xsd",
    "redirect-urls" : "redirecturl.xsd",
    "schedules" : "schedules.xsd",
    "search" : "search.xsd",
    "search2" : "search2.xsd",
    "services" : "services.xsd",
    "shipping" : "shipping.xsd",
    "site" : "site.xsd",
    "slot-configurations" : "slot.xsd",
    "sort" : "sort.xsd",
    "sourcecodes" : "sourcecode.xsd",
    "stores" : "store.xsd",
    "tax" : "tax.xsd",
    "url-rules" : "urlrules.xsd",
    "xml" : "xml.xsd"
}

def validate_xml(filename):
    xml = ET.parse(filename)
    root_el = xml.getroot()
    root_tag = root_el.tag[root_el.tag.find('}')+1:]
    if root_tag not in SCHEMA_MAP:
        raise Exception("cannot find schema for %s" % root_tag)
    schema_name = SCHEMA_MAP[root_tag]
    schema = ET.XMLSchema(file=os.path.join(dwre_tools.__path__[0], 'schemas', schema_name))
    schema.assertValid(xml)


In [369]:
validate_xml("/Users/clavery/Downloads/05182015-ContentAssets/libraries/VibramGlobalSharedLibrary/library.xml")


## Bootstrap

In [387]:
TOOL_VERSION = u"4"
BOOTSTRAP_META = """<?xml version="1.0" encoding="UTF-8"?>
<metadata xmlns="http://www.demandware.com/xml/impex/metadata/2006-10-31">
    <type-extension type-id="OrganizationPreferences">
        <custom-attribute-definitions>
            <attribute-definition attribute-id="dwreMigrateCurrentVersion">
                <display-name xml:lang="x-default">DWRE Migrate Current Version</display-name>
                <description>DO NOT MODIFY THIS VALUE UNLESS YOU UNDERSTAND THE CONSEQUENCES. PERFORM A SITE BACKUP BEFORE MANUAL MODIFICATION</description>
                <type>string</type>
                <site-specific-flag>false</site-specific-flag>
                <mandatory-flag>false</mandatory-flag>
                <externally-managed-flag>true</externally-managed-flag>
                <min-length>0</min-length>
            </attribute-definition>
            <attribute-definition attribute-id="dwreMigrateToolVersion">
                <display-name xml:lang="x-default">DWRE Migrate Tool Version</display-name>
                <description>DO NOT MODIFY THIS VALUE UNLESS YOU UNDERSTAND THE CONSEQUENCES. PERFORM A SITE BACKUP BEFORE MANUAL MODIFICATION</description>
                <type>string</type>
                <site-specific-flag>false</site-specific-flag>
                <mandatory-flag>false</mandatory-flag>
                <externally-managed-flag>true</externally-managed-flag>
                <min-length>0</min-length>
                <default-value>1</default-value>
            </attribute-definition>
        </custom-attribute-definitions>
        <group-definitions>
            <attribute-group group-id="dwreMigrate">
                <display-name xml:lang="x-default">DWREMigrate</display-name>
                <attribute attribute-id="dwreMigrateCurrentVersion"/>
                <attribute attribute-id="dwreMigrateToolVersion"/>
            </attribute-group>
        </group-definitions>
    </type-extension>
</metadata>
"""

In [388]:
PREFERENCES = """<?xml version="1.0" encoding="UTF-8"?>
<preferences xmlns="http://www.demandware.com/xml/impex/preferences/2007-03-31">
    <custom-preferences>
        <all-instances>
            <preference preference-id="dwreMigrateToolVersion">%s</preference>
        </all-instances>
    </custom-preferences>
</preferences>
""" % (TOOL_VERSION)

VERSION = """###########################################
# Generated file, do not edit.
# Copyright (c) 2015 by Demandware, Inc.
###########################################
15.5.2
"""

In [389]:
dest_file = "DWREMigrateBootstrap_v{}".format(TOOL_VERSION)

In [390]:
bootstrap_package_file = io.BytesIO()
bootstrap_package_zip = zipfile.ZipFile(bootstrap_package_file, "w")

In [391]:
bootstrap_package_zip.writestr("{}/version.txt".format(dest_file), VERSION)
bootstrap_package_zip.writestr("{}/preferences.xml".format(dest_file), PREFERENCES.encode("utf-8"))
bootstrap_package_zip.writestr("{}/meta/system-objecttype-extensions.xml".format(dest_file), BOOTSTRAP_META)
bootstrap_package_zip.close()

In [298]:
with open("test.zip", "wb") as f:
    bootstrap_package_file.seek(0)
    f.write(bootstrap_package_file.read())

### Upload Bootstrap

In [191]:
dest_url = "https://{0}/on/demandware.servlet/webdav/Sites/Impex/src/instance/{1}.zip".format(
    ENV["server"], dest_file)
webdavsession.put(dest_url, data=bootstrap_package_file)

<Response [201]>

### Activate Bootstrap

In [192]:
response = bmsession.post("https://{}/on/demandware.store/Sites-Site/default/ViewSiteImpex-Dispatch".format(ENV["server"]),
                          data={"import" :"", "ImportFileName" : dest_file + ".zip", "realmUse": "False"})

In [193]:
wait_for_import()

### Delete Impex Files

In [194]:
dest_url = "https://{0}/on/demandware.servlet/webdav/Sites/Impex/src/instance/{1}.zip".format(
    ENV["server"], dest_file)
webdavsession.delete(dest_url)

<Response [204]>

## Load Migrations Context

### Query Versions

In [395]:
versions_url = "https://{}/on/demandware.store/Sites-Site/default/DWREMigrate-Versions".format(ENV["server"])
response = bmsession.get(versions_url)

if "application/json" not in response.headers['content-type']:
    print "Error"
else:
    tool_version = response.json()["toolVersion"]
    migration_version = response.json()["migrationVersion"]
    bootstrap_required = response.json()["missingToolVersion"]
    if int(TOOL_VERSION) > int(tool_version):
        print "Upgrade Needed", TOOL_VERSION, tool_version
    else:
        print "Tool version up to date"
        

Tool version up to date


## Determine Required Migrations

In [396]:
def find_path(graph, start, path=None):
    if path is None:
        path = []
        
    path = path + [start]

    if not graph.has_key(start):
        return path
    
    for node in graph[start]:
        if node not in path:
            newpath = find_path(graph, node, path)
            if newpath: return newpath
            else: return path
    return None

In [397]:
migrations = []
X = "{http://www.pixelmedia.com/xml/dwremigrate}"

if int(TOOL_VERSION) > int(tool_version):
    migrations.append({"id" : "bootstrap", "location" : bootstrap_package_file, "description" : "DWRE Migrate Tool Upgrade"})
elif bootstrap_required:
    migrations.append({"id" : "bootstrap", "location" : bootstrap_package_file, "description" : "DWRE Migrate Bootstrap"})
    
    
migrations_file = os.path.join(MIGRATIONS_DIR, "migrations.xml")
assert os.path.exists(migrations_file), "Cannot find migrations.xml"
migrations_context = ET.parse(migrations_file)

# validate migrations
schema = ET.XMLSchema(file=os.path.join(dwre_tools.__path__[0], 'schemas', 'dwre-migrate.xsd'))
if not schema.validate(migrations_context):
    print "Migrations context not valid"
    print schema.error_log

migration_nodes = defaultdict(list)
migration_data = {}
for migration in migrations_context.getroot():
    id = migration.attrib["id"]
    location = migration.find(X + "location").text
    description_el = migration.find(X + "description")
    parent_el = migration.find(X + "parent")
    
    description = ""
    if description_el is not None:
        description = description_el.text
        
    migration_data[id] = {"id": id, "location" : location, "description": description}
    if parent_el is not None:
        migration_nodes[parent_el.text].append(id)
    else:
        migration_nodes[None].append(id)
    
# validate root exists
assert migration_nodes[None], "Cannot find root migration (migration with no parent)"

errors = []
# validate graph structure
for parent, names in migration_nodes.items():
    if parent and parent not in migration_data:
        errors.append("Cannot find migration: {}".format(parent))
    if len(names) != 1:
        errors.append("Migration ({}) has multiple ({}) children".format(
                      parent if parent else "ROOT", [m for m in names]))

assert migration_version is None or migration_version in migration_data, "Cannot find migration %s; requires manual intervention" % migration_version

if errors:
    print "Errors in migration context: "
    for e in errors:
        print e
else:
    path = []
    if migration_nodes[migration_version]:
        path = find_path(migration_nodes, migration_nodes[migration_version].pop())
    migrations.extend([migration_data[m] for m in path])

In [398]:
if migrations:
    print "%s migrations required..." % len(migrations)
else:
    print "No migrations required. Instance is up to date: %s" % migration_version
for migration in migrations:
    print "[%s] %s" % (migration["id"], migration["description"])
 
migrate = "test"
if migrate:
    pass

3 migrations required...
[2015-07-10_initial] Initial SWK Site Migration
[2015-07-10_m2] m2 description
[2015-07-10_m3] 


## Perform Migrations

In [17]:
def directory_to_zip(directory, filename):
    zip_file_io = io.BytesIO()
    zip_file = zipfile.ZipFile(zip_file_io, "w")

    for (dirpath, dirnames, filenames) in os.walk(directory):
        basepath = dirpath[len(directory)+1:]
        for fname in filenames:
            print "fname", fname
            print "basepath", basepath
            zip_file.write(os.path.join(dirpath, fname), os.path.join(filename, basepath, fname))
    zip_file.close()
    return zip_file_io

In [400]:
for migration in migrations:
    start_time = time.time()
    zip_filename = "dwremigrate_%s" % migration["id"]
    if isinstance(migration["location"], io.BytesIO):
        zip_file = migration["location"]
        zip_filename = "DWREMigrateBootstrap_v{}".format(TOOL_VERSION)
    else:
        zip_file = directory_to_zip(os.path.join(MIGRATIONS_DIR, migration["location"]), zip_filename)
    
    # upload
    dest_url = "https://{0}/on/demandware.servlet/webdav/Sites/Impex/src/instance/{1}.zip".format(
    ENV["server"], zip_filename)
    webdavsession.put(dest_url, data=zip_file)
    
    # activate
    response = bmsession.post("https://{}/on/demandware.store/Sites-Site/default/ViewSiteImpex-Dispatch".format(ENV["server"]),
                          data={"import" :"", "ImportFileName" : zip_filename + ".zip", "realmUse": "False"})
    
    wait_for_import(bmsession, zip_filename)
    
    # update migration version
    if migration["id"] != "bootstrap":
        response = bmsession.post("https://{}/on/demandware.store/Sites-Site/default/DWREMigrate-UpdateVersion".format(ENV["server"]),
                                  data={"NewVersion" : migration["id"]})
    
    # delete file
    dest_url = "https://{0}/on/demandware.servlet/webdav/Sites/Impex/src/instance/{1}.zip".format(
                ENV["server"], zip_filename)
    webdavsession.delete(dest_url)
    
    end_time = time.time()
    print "Migrated %s in %.3f seconds" % (migration["id"], end_time - start_time)

Migrated 2015-07-10_initial in 1.228 seconds
Migrated 2015-07-10_m2 in 0.843 seconds
Migrated 2015-07-10_m3 in 0.863 seconds


## Client Certificate

In [21]:
webdavsession = requests.session()
webdavsession.auth=("clavery", "JG7hlmrLGgXttQu1!",)
webdavsession.cert=("build.crt", "build_private.key",)
webdavsession.verify=False

test_zip = directory_to_zip("./DWREMigrateBootstrap", "test")
webdavsession.put("https://cert.staging.web.stonewall.demandware.net/on/demandware.servlet/webdav/Sites/Impex/src/instance/test.zip",
                 data=test_zip)

fname .DS_Store
basepath 
fname preferences.xml
basepath 
fname version.txt
basepath 
fname system-objecttype-extensions.xml
basepath meta


/Users/clavery/.venv/default/lib/python2.7/site-packages/requests/packages/urllib3/connectionpool.py:768: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)


<Response [204]>

In [20]:
webdavsession.verify

True